Gather necessary pre-requisites:

In [1]:
# import everything you need
from nipype import Node, Workflow
import nipype.interfaces.fsl as fsl
#import nipype.interfaces.spm as spm
#import nibabel as nb
#from nilearn.plotting import plot_anat
import numpy as np
import pandas as pd
# activate inline magics
%matplotlib inline 
import matplotlib.pyplot as plt
import os, operator, re, json, random
from functools import reduce

# General

Write useful functions

In [10]:
# write update function for file list to data dictionary
def update_files():
    for sub in FEDs:
        # collect image files from source dirs
        files=sorted([os.path.join(subdir, content)
                      for subdir in FED_dirs
                      for content in os.listdir(subdir)
                      if re.match(r'(.*.(nii|json))', content)])
        # extend content
        content=[file for file in files
                 if re.match(fr'(.*{sub}.*)', file)]
        # update files in data
        data[sub]["files"]=content

Get relevant files

In [11]:
# define base diectory for fMRI folder structure and FEDs
basedir="/fMRI/"
FED_dirs=[os.path.join(basedir, FED) 
          for FED in os.listdir(basedir)]

In [26]:
# collect image files
files=sorted([os.path.join(subdir, content)
       for subdir in FED_dirs
       for content in os.listdir(subdir)
       if re.match(r'(.*.(nii|json))', content)])
# create a dictionary storing all files/subject
data={}
for fed in FED_dirs:
    ID=fed.rsplit('/', 1)[1]
    # define partition for each subject
    subject={"files":[], "parameters":{}}
    content=[]
    for file in files:
        if re.match(fr'(.*{ID}.*)', file):
            content.append(file)
    subject["files"].extend(content)
    data[ID]=subject
    
# create shortcut to FEDs
FEDs=[key for key in data.keys()]

Get relevant parameters

In [27]:
# define exclusion set for FEDs that do not meet the requirements
GRE_excluded=[]
EPI_excluded=[]

# FED control
for subject in FEDs:
    # collect relevant files/FED for parameter extraction
    GREs=[file for file in data[subject]["files"]
           if re.match(r'(.*(_e1|_e2(?!_ph)).*.json)', file)]
    EPIs=[file for file in data[subject]["files"]
           if re.match(r'(.*(FMRI).*.json)', file)]
    # control number of files
    # There seem to be several omissions -> investigate
    if len(GREs) < 2:
        print(subject, "\n", f"Not enough GRE_FIELD sequences to read out ({len(GREs)}).... exclude/investigate", "\n")
        GRE_excluded.append(subject)
      
    # if there is more than one pair of magnitude images (e1+e2),
    # take the first one as they are likely to be closer to the fMRI acquisition
    elif len(GREs) > 2:
        print(subject, "\n", "More than 2 GRE_FIELD sequences",
              "("+str(len(GREs))+")", ".... select only first two for TE extraction", "\n")
    
    elif len(EPIs) != 1:
            print(subject, "\n", f"Not exactly one FMRI sequence to read out ({len(EPIs)}).... investigate", "\n")
            EPI_excluded.append(subject)

# exclude FEDs from distortion correction
print("\n\n", "The following subjects where excluded from distortion correction due to false file numbers: ",\
      "\n", "GRE: ", sorted(GRE_excluded), "EPI: ", sorted(EPI_excluded))

# make new list for calculating fieldmaps
FEDs=[sub for sub in FEDs if sub not in GRE_excluded and sub not in EPI_excluded]

FED047 
 Not enough GRE_FIELD sequences to read out (0).... exclude/investigate 

FED037 
 Not enough GRE_FIELD sequences to read out (0).... exclude/investigate 

FED006 
 Not enough GRE_FIELD sequences to read out (0).... exclude/investigate 

FED022 
 More than 2 GRE_FIELD sequences (4) .... select only first two for TE extraction 

FED067 
 Not enough GRE_FIELD sequences to read out (0).... exclude/investigate 

FED008 
 Not enough GRE_FIELD sequences to read out (0).... exclude/investigate 

FED023 
 More than 2 GRE_FIELD sequences (4) .... select only first two for TE extraction 

FED029 
 Not enough GRE_FIELD sequences to read out (0).... exclude/investigate 



 The following subjects where excluded from distortion correction due to false file numbers:  
 GRE:  ['FED006', 'FED008', 'FED029', 'FED037', 'FED047', 'FED067'] EPI:  []


In [28]:
# Read json info data for all relevant parameters
# define info of interest
GREspecs=["EchoTime", "PhaseEncodingDirection"]
EPIspecs=["EchoTime", "RepetitionTime", "EchoTrainLength", 
          "PhaseEncodingSteps", "PhaseEncodingDirection",
          "DwellTime", "TotalReadoutTime", "EffectiveEchoSpacing", "PixelBandwidth"]
# create list to record missing parameters
missing_params=[]

# Now for the parameter extraction
for subject in FEDs:
    # collect relevant files/FED for parameter extraction
    GREs=[file for file in data[subject]["files"]
           if re.match(r'(.*(_e1|_e2(?!_ph)).*.json)', file)]
    EPIs=[file for file in data[subject]["files"]
           if re.match(r'(.*(FMRI).*.json)', file)]
    
    # if there is more than one pair of magnitude images (e1+e2),
    # take the first one as they are likely to be closer to the fMRI acquisition
    if len(GREs) > 2:
        print(subject, "\n", "More than 2 GRE_FIELD sequences",
              "("+str(len(GREs))+")", ".... selecting first two for TE extraction", "\n")
        GREs = GREs[0:2] 
    
    # GRE PARAMETERS
    # collect parameters from the respective echoe's json file
    for file in GREs:
        with open(file) as json_file:
            info=json.load(json_file)
            for param in GREspecs:
                # put params into data; use capital letters of parameter name as indicator in key
                key=f"GRE_{''.join([char for char in param if char.isupper()])}"
                # if key does not exist -> create list with parameter
                if key not in data[subject]["parameters"].keys():
                    try: data[subject]["parameters"][key]=[info[param]]
                    # if parameter does not exist in json file -> missing_params
                    # append to GRE_excluded
                    except KeyError:
                        print(f"{subject}'s json file does not specify {param}", "\n",
                             "noting issue ... ")
                        missing_params.append(f"{subject}_{param}")
                        GRE_excluded.append(subject)
                        pass
                # if key does exist -> append parameter to the list
                if key in data[subject]["parameters"].keys():
                    if info[param] not in data[subject]["parameters"][key]:
                        data[subject]["parameters"][key].append(info[param])
                    elif info[param] in data[subject]["parameters"][key]:
                        pass

    # EPI PARAMETERS
    # collect parameters from the respective scans
    for file in EPIs:
        with open(file) as json_file:
            info=json.load(json_file)
            for param in EPIspecs:
                # put params into data; use capital letters of parameter name as indicator in key
                key=f"EPI_{''.join([char for char in param if char.isupper()])}"
                # if key does not exist -> create list with parameter
                if key not in data[subject]["parameters"].keys():
                    try: data[subject]["parameters"][key]=[info[param]]
                    # if parameter does not exist in json file -> missing_params
                    # append to EPI_excluded
                    except KeyError:
                        print(f"{subject}'s json file does not specify {param}", "\n",
                             "noting issue ... ")
                        missing_params.append(f"{subject}_{param}")
                        EPI_excluded.append(subject)
                        pass
                # if key does exist -> append to the list
                if key in data[subject]["parameters"].keys():
                    if info[param] not in data[subject]["parameters"][key]:
                        data[subject]["parameters"][key].append(info[param])
                    elif info[param] in data[subject]["parameters"][key]:
                        pass

# show missing parameters from json files:
print("The following parameters where not available from subjects' json files:", 
      "\n",sorted(missing_params))

# exclude FEDs from distortion correction
print("\n\n", "The following subjects where excluded from distortion correction due to flase file numbers or missing parameters: ",\
      "\n", "GRE: ", sorted(GRE_excluded), "EPI: ", sorted(EPI_excluded), "\n",
     len(GRE_excluded)+len(EPI_excluded), "  subjects in total")

# make new list for calculating fieldmaps
FEDs=[sub for sub in FEDs if sub not in GRE_excluded and sub not in EPI_excluded]

FED015
FED012
FED030
FED054
FED042
FED024
FED038
FED045
FED061
FED035
FED022
FED022 
 More than 2 GRE_FIELD sequences (4) .... selecting first two for TE extraction 

FED009
FED036
FED019
FED017
FED053
FED043
FED055
FED058
FED066
FED066's json file does not specify DwellTime 
 noting issue ... 
FED021
FED056
FED027
FED020
FED048
FED026
FED016
FED013
FED031
FED039
FED025
FED023
FED023 
 More than 2 GRE_FIELD sequences (4) .... selecting first two for TE extraction 

FED044
FED057
FED064
FED064's json file does not specify DwellTime 
 noting issue ... 
FED010
FED065
FED065's json file does not specify DwellTime 
 noting issue ... 
FED068
FED014
FED033
FED063
FED063's json file does not specify DwellTime 
 noting issue ... 
FED050
FED040
FED062
FED052
FED034
FED007
FED041
FED028
FED011
FED059
FED049
FED046
FED032
FED018
FED051
FED060
The following parameters where not available from subjects' json files: 
 ['FED063_DwellTime', 'FED064_DwellTime', 'FED065_DwellTime', 'FED066_DwellTime']




In [29]:
# Edit the read-in parameters into analysis format
for sub in sorted(FEDs):
    # calculate necessary parameters that are not in header information
    # deltaTE
    data[subject]["parameters"]["DeltaTE"]= reduce(operator.sub, data[subject]["parameters"]["GRE_ET"])*-1*1000
    
    # transfer phase encoding directions from field axes to voxel axes
    # control field axes values
#    print(sub)
#    print(data[sub]["parameters"]["EPI_PED"][0])
    epi_phasecodedir=data[sub]["parameters"]["EPI_PED"][0]
    for char in epi_phasecodedir:
        if char == "i":
            data[sub]["parameters"]["EPI_PED"][0]=f"x{epi_phasecodedir[1:]}"
        elif char == "j":
            data[sub]["parameters"]["EPI_PED"][0]=f"y{epi_phasecodedir[1:]}"
        elif char == "k":
            data[sub]["parameters"]["EPI_PED"][0]=f"z{epi_phasecodedir[1:]}"
    # control new values
#    print(data[sub]["parameters"]["EPI_PED"][0])

FED007
j-
y-
FED009
j-
y-
FED010
j-
y-
FED011
j-
y-
FED012
j-
y-
FED013
j-
y-
FED014
j-
y-
FED015
j-
y-
FED016
j-
y-
FED017
j-
y-
FED018
i
x
FED019
j-
y-
FED020
i
x
FED021
i
x
FED022
i
x
FED023
i
x
FED024
i
x
FED025
i
x
FED026
j-
y-
FED027
j-
y-
FED028
j-
y-
FED030
j-
y-
FED031
j-
y-
FED032
j-
y-
FED033
i
x
FED034
j-
y-
FED035
j-
y-
FED036
j-
y-
FED038
j-
y-
FED039
j-
y-
FED040
j-
y-
FED041
j-
y-
FED042
j-
y-
FED043
j-
y-
FED044
j-
y-
FED045
j-
y-
FED046
j-
y-
FED048
j-
y-
FED049
j-
y-
FED050
j-
y-
FED051
j-
y-
FED052
j-
y-
FED053
j-
y-
FED054
j-
y-
FED055
j-
y-
FED056
j-
y-
FED057
j-
y-
FED058
j-
y-
FED059
j-
y-
FED060
j-
y-
FED061
j-
y-
FED062
j-
y-
FED068
j-
y-


In [24]:
# control results in a random fashion
key= random.sample(data.keys(), 1)
print(data[key[0]])
print(data["FED012"])

{'files': ['/fMRI/FED067/FMRI_0007_20160311133810_7.json', '/fMRI/FED067/FMRI_0007_20160311133810_7.nii', '/fMRI/FED067/T1_MPRAGE_0005_20160311133810.json', '/fMRI/FED067/T1_MPRAGE_0005_20160311133810.nii'], 'parameters': {}}
{'files': ['/fMRI/FED012/FMRI_FS_0010_20141124150148_10.json', '/fMRI/FED012/FMRI_FS_0010_20141124150148_10.nii', '/fMRI/FED012/GRE_FIELD_MAPPING_0006_20141124150148_e1.json', '/fMRI/FED012/GRE_FIELD_MAPPING_0006_20141124150148_e1.nii', '/fMRI/FED012/GRE_FIELD_MAPPING_0006_20141124150148_e2.json', '/fMRI/FED012/GRE_FIELD_MAPPING_0006_20141124150148_e2.nii', '/fMRI/FED012/GRE_FIELD_MAPPING_0007_20141124150148_e2_ph.json', '/fMRI/FED012/GRE_FIELD_MAPPING_0007_20141124150148_e2_ph.nii', '/fMRI/FED012/GRE_e1mag_bet_B.nii', '/fMRI/FED012/GRE_e1mag_bet_B_mask.nii', '/fMRI/FED012/GRE_fieldmap.nii', '/fMRI/FED012/T1_MPRAGE_0005_20141124150148.json', '/fMRI/FED012/T1_MPRAGE_0005_20141124150148.nii'], 'parameters': {'GRE_ET': [0.00492, 0.00738], 'GRE_PED': ['i'], 'EPI_ET': 

# Processing

Motion correct / realign

Slicetime correction (?)

I will omit this step as advised by Poldrack, Nichols and Mumford "Handbook of fMRI data analysis"(2011)\
TR= 1800ms  > 2seconds, interleaved acquisition\
event related analysis is relatively robust to slice-timing problems here.

Spatial normalisation

Spatial Smoothing

# QC